In [69]:
from BeautifulSoup import BeautifulSoup
import re, string
import json

def parse_json_stream(stream):
    decoder = json.JSONDecoder()
    while stream:
        obj, idx = decoder.raw_decode(stream)
        yield obj
        stream = stream[idx:].lstrip()

with open('linkedin_html.json', 'r') as f:
    read_data = parse_json_stream(f.readlines()[0])


In [70]:
from datetime import datetime
def parse_datetime(datestring):
    x = sorted(datestring.split())
    if not x:
        return ''
    if len(x) == 1:
        return str(datetime.strptime(x[0], '%Y'))
    return str(datetime.strptime(' '.join(x), '%Y %B'))
        

In [71]:
def clean_str(text):
    return ''.join([i if ord(i) < 128 else ' ' for i in text])

In [72]:
def return_date(html):
    dates = html.findAll('time')
    dates = [x.string for x in dates]
    dates = [clean_str(x) for x in dates]
    dates = [parse_datetime(x) for x in dates]
    
    datestr = html.find('span',{'class':'experience-date-locale'})
    if datestr and len(dates) == 1:
        if 'present' in datestr.text.lower():
            dates.append(str(datetime.today()))
    
    return tuple(dates)

In [73]:
def find_ed(soup):
    education = soup.findAll('div', {'class':re.compile("(^education)")})
    all_ed = []
    for item in education:
        ed_dict = {}
        head = item.find('header')
        
        ed_dict['institution'] = ''.join(head.h4.findAll(text=True))        
        ed_dict['degree'] = ''.join(head.h5.findAll(text=True))
        ed_dict['dates'] = return_date(item)

        all_ed.append(ed_dict)
    return all_ed



In [74]:
def find_exp(soup):
    experiences = soup.findAll('div', {'id':re.compile("(^experience.*-view)")})
    all_exp = []
    
    for item in experiences:
        exp_dict = {}
        exp_dict['title'] = item.find('a', {'title':'Learn more about this title'}).string
        exp_dict['organization'] = item.find('h5', attrs={'class': None}).next.string
        exp_dict['dates'] = return_date(item)
        
        if item.find('span', {'class': 'locality'}):
            exp_dict['locality'] = item.find('span', {'class': 'locality'}).string   

        all_exp.append(exp_dict)
    return all_exp

In [75]:
import sys

linkedin_data = {}

for idx, profile in enumerate(read_data):    
    
    sys.stdout.write("\r%d" % idx)
    sys.stdout.flush()
    
    name, html = profile[:]
    soup = BeautifulSoup(html)
    person = {}
    person['edu'] = find_ed(soup)
    person['exp'] = find_exp(soup)
    link = soup.findAll('link', {'rel':'canonical'})
    if len(link)>0:
        person['link'] = link[0]['href']
    linkedin_data[name] = person

422

In [76]:
import json

with open('linkedin_data2.json','w') as f:
    json.dump(linkedin_data, f)